#### Libraries importeren

In [2]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import torch.nn as nn 
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

#### Data inladen en naar dataframe omzetten

In [3]:
data = json.load(open('Data/Intent.json'))
df = pd.DataFrame(data['intents'])
df = df[['intent', 'text', 'responses']]
df.head()

,intent,text,responses
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, ...","[Hi human, please tell me your GeniSys user, H..."
1,GreetingResponse,"[My user is Adam, This is Adam, I am Adam, It ...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ..."
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are ...","[Hello, I am great, how are you? Please tell m..."
3,CourtesyGreetingResponse,"[Good thanks! My user is Adam, Good thanks! Th...","[Great! Hi <HUMAN>! How can I help?, Good! Hi ..."
4,CurrentHumanQuery,"[What is my name?, What do you call me?, Who d...","[You are <HUMAN>! How can I help?, Your name i..."


#### Functies maken voor tokenizen

In [17]:
nltk.download('punkt_tab')
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

def stem (word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1.0
    return bag

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\boris\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


#### Woordenlijst en trainingdata maken

In [19]:
all_words = []
tags = []
xy = []

for _, intent in df.iterrows():
    tag = intent["intent"]
    tags.append(tag)
    
    for pattern in intent["text"]:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
        
all_words = sorted(set(stem(w) for w in all_words if w not in ["?", ".", "!"]))
tags = sorted(set(tags))

In [ ]:
X_train = []
Y_train = []

for pattern_sentence, tag in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    Y_train.append(tags.index(tag))
    
X_train = np.array(X_train)
Y_train = np.array(Y_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]], dtype=float32)